In [345]:
import re
import pythainlp
from pythainlp import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
import os
import pandas as pd
from pythainlp.corpus import wordnet
from pythainlp.corpus.common import thai_stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings

In [346]:
warnings.filterwarnings("ignore")
nltk.download('words')
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to /Users/admin/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [347]:
train_column_names=['reviews', 'rating']
test_column_names=['id', 'reviews']
df_train = pd.read_csv('w_review_train.csv', sep=',', header=None,names=train_column_names)
df_test = pd.read_csv('test_file.csv', sep=';', header=None,names=test_column_names)
target = df_train['rating']

In [348]:
df_train

,reviews,rating
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [349]:
df_test = df_test[:300]

## Clean data

In [350]:

custom_th_stop = ('อาทิ', 'ดร', 'มรว', 'มล', 'พตต', 'พตท', 'พตอ',) 
th_stop = tuple(thai_stopwords()) + custom_th_stop
def preprocess_raw_text(text):
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]
    return tokens

In [351]:
reviews_train_clean = []
for index,row in df_train.iterrows():
    reviews_train_clean.append(preprocess_raw_text(row['reviews']))
    
reviews_test_clean = []
for index,row in df_test.iterrows():
    reviews_test_clean.append(preprocess_raw_text(row['reviews']))    

## Word Counts

In [352]:
from sklearn.feature_extraction.text import CountVectorizer
reviews_train_clean_j = [','.join(tkn) for tkn in reviews_train_clean]
reviews_test_clean_j = [','.join(tkn) for tkn in reviews_test_clean]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
cvec.fit(reviews_train_clean_j)
X= cvec.transform(reviews_train_clean_j)
X_test= cvec.transform(reviews_test_clean_j)

In [343]:
# count word uncomment below code
cvec.vocabulary_

{'ร้านอาหาร': 3198,
 'กก': 904,
 '\n': 1,
 'เลี้ยว': 4521,
 'เข้ามา': 4109,
 'เจอ': 4154,
 'ห้องน้ำ': 3811,
 'เออ': 4646,
 'แปลก': 4803,
 'ดี': 1780,
 'ห้อง': 3810,
 'ทาน': 2109,
 'หลัก': 3727,
 'ชั้น': 1584,
 'กาแฟ': 1027,
 'น้ำผึ้ง': 2341,
 'ราด': 3122,
 'แพง': 4821,
 'เวอร์': 4549,
 'อย่า': 3849,
 'สั่ง': 3545,
 'ลาบ': 3255,
 'ไข่ต้ม': 5122,
 'ไข่': 5121,
 'คาว': 1375,
 'อะ': 3894,
 'ประทับใจ': 2476,
 'ทอดมัน': 2068,
 'หัวปลี': 3766,
 'ล้า': 3306,
 'อร่อย': 3870,
 'เบิ้ล': 4325,
 'พะแนง': 2712,
 'ห่อ': 3804,
 'เห้ย': 4638,
 'ค่า': 1429,
 'บ.': 2361,
 'เกิน': 4069,
 'ไม่ไหว': 5218,
 'ว': 3312,
 'เลิก': 4512,
 'กิน': 1038,
 'ขนมหวาน': 1107,
 'ฟรี': 2777,
 ')': 31,
 'ขนมไทย': 1110,
 'ซ้ำ': 1726,
 'อาหาร': 3926,
 'ที่นี่': 2148,
 'อาหารจีน': 3929,
 'แคะ': 4718,
 'หากิน': 3778,
 'บ้าน': 2447,
 'ตัว': 1904,
 'ร้าน': 3197,
 'ตั้งอยู่': 1917,
 'ถ.': 2012,
 'พุทธมณฑล': 2763,
 'สาย': 3560,
 'ปิ่น': 2554,
 'เกล้า': 4061,
 '-': 45,
 'นครชัยศรี': 2209,
 'เข้าไป': 4113,
 'ม.': 2827,
 'ซ้ายมือ': 1

In [356]:
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75, 
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_wc = LogisticRegression(C=1)
final_wc.fit(X, target)
print("Final Accuracy: %s" % accuracy_score(target, final_wc.predict(X_test)))

Accuracy for C=0.01: 0.49333333333333335
Accuracy for C=0.05: 0.48
Accuracy for C=0.25: 0.5333333333333333
Accuracy for C=0.5: 0.5333333333333333
Accuracy for C=1: 0.5333333333333333
Final Accuracy: 0.51


## TF-IDF

In [354]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(reviews_train_clean_j)
X= tfidf_vectorizer.transform(reviews_train_clean_j)
X_test= tfidf_vectorizer.transform(reviews_test_clean_j)
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

final_tfidf = LogisticRegression(C=1)
final_tfidf.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_tfidf.predict(X_test)))


Accuracy for C=0.01: 0.4666666666666667
Accuracy for C=0.05: 0.4666666666666667
Accuracy for C=0.25: 0.4666666666666667
Accuracy for C=0.5: 0.4533333333333333
Accuracy for C=1: 0.4666666666666667
Final Accuracy: 0.52


## SVM


In [355]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean_j)
X= ngram_vectorizer.transform(reviews_train_clean_j)
X_test= ngram_vectorizer.transform(reviews_test_clean_j)
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))

final_svm_ngram = LinearSVC(C=0.01)
final_svm_ngram.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_svm_ngram.predict(X_test)))


Accuracy for C=0.01: 0.4666666666666667
Accuracy for C=0.05: 0.48
Accuracy for C=0.25: 0.44
Accuracy for C=0.5: 0.44
Accuracy for C=1: 0.4266666666666667
Final Accuracy: 0.51
